In [1]:
#Libraries used
import pandas as pd #for the data frame
import numpy as np #used for filtering and some arithmatic. 

In [2]:
#Import the data
df_import = pd.read_csv('data/pbp/raw/pbp_query_20162017.csv',low_memory=False) 

In [3]:
#The first thing to do is reduce the number of columns then rows that won't be used
df = df_import[['season', 'game_id', 'game_date', 'event_index', 'game_period',
                'game_seconds', 'event_type', 'event_description', 'event_detail',
                'event_team', 'coords_x', 'coords_y', 'home_team', 'away_team',
                'home_score', 'away_score', 'game_strength_state', 'event_distance',
                'event_angle']]
even_strength = df.game_strength_state.isin(['5v5','4v4','3v3'])
play_type     = df.event_type.isin(['FAC','SHOT','HIT','BLOCK','MISS','GIVE','TAKE','GOAL'])
no_coords     = ~df.coords_x.isna()
df            = df[even_strength & play_type & no_coords]

In [55]:
away_flip = (df.event_team == df.away_team) & (df.game_period.isin([1,3]))
home_flip = (df.event_team == df.home_team) & (df.game_period == 2)
df['x_flip'] = np.where(away_flip | home_flip,-1*df.coords_x,df.coords_x)
df['y_flip'] = np.where(away_flip | home_flip,-1*df.coords_y,df.coords_y)

In [56]:
df.sample(10)

,season,game_id,game_date,event_index,game_period,game_seconds,event_type,event_description,event_detail,event_team,...,coords_y,home_team,away_team,home_score,away_score,game_strength_state,event_distance,event_angle,x_flip,y_flip
44687,20162017,2016020147,2016-11-03,619,3,3596,HIT,"NYI #15 CLUTTERBUCK HIT PHI #23 MANNING, Off. ...",NaN,NYI,...,39.0,NYI,PHI,2,2,5v5,45.79,58.39,65.0,39.0
81181,20162017,2016020267,2016-11-19,502,3,2854,SHOT,"STL ONGOAL - #12 LEHTERA, Wrist, Off. Zone, 30...",Wrist,STL,...,10.0,STL,NSH,2,1,5v5,29.73,19.65,61.0,10.0
224126,20162017,2016020738,2017-01-26,617,3,3371,MISS,"BUF #90 O'REILLY, Slap, Over Net, Off. Zone, 4...",Slap,BUF,...,29.0,DAL,BUF,4,3,5v5,45.45,39.64,-54.0,-29.0
85263,20162017,2016020281,2016-11-21,95,1,499,MISS,"CHI #48 HINOSTROZA, Deflected, Wide of Net, Of...",Deflected,CHI,...,-8.0,EDM,CHI,1,0,5v5,9.43,57.99,84.0,8.0
265772,20162017,2016020875,2017-02-19,568,3,2942,FAC,CAR won Neu. Zone - TOR #42 BOZAK vs CAR #11 S...,NaN,CAR,...,-22.0,CAR,TOR,0,4,5v5,72.42,17.68,-20.0,-22.0
301002,20162017,2016020992,2017-03-09,631,3,3200,TAKE,"VAN TAKEAWAY - #50 GAUNCE, Off. Zone",NaN,VAN,...,38.0,VAN,NYI,2,3,5v5,46.04,55.62,-63.0,38.0
234380,20162017,2016020772,2017-02-04,331,2,1614,HIT,"PHI #13 LYUBIMOV HIT L.A #10 SETOGUCHI, Def. Zone",NaN,PHI,...,-38.0,PHI,L.A,0,0,5v5,43.91,59.93,67.0,38.0
301377,20162017,2016020994,2017-03-09,11,1,44,HIT,"WSH #27 ALZNER HIT S.J #12 MARLEAU, Def. Zone",NaN,WSH,...,-18.0,S.J,WSH,0,0,5v5,19.70,66.04,-97.0,18.0
153960,20162017,2016020506,2016-12-23,411,2,2316,BLOCK,MIN #46 SPURGEON BLOCKED BY NYR #4 CLENDENING...,Wrist,MIN,...,23.0,NYR,MIN,2,6,5v5,29.21,51.95,-71.0,23.0
249277,20162017,2016020821,2017-02-11,530,3,2615,BLOCK,"TOR #12 BROWN BLOCKED BY BUF #15 EICHEL, Snap...",Snap,TOR,...,9.0,TOR,BUF,1,3,5v5,26.57,19.80,-64.0,9.0


In [64]:
check_lst = ['coords_x','coords_y','x_flip','y_flip',
             'event_distance','event_angle','event_team','home_team']
df_rc = df[check_lst].copy()

In [65]:
df_rc.sample(10)

,coords_x,coords_y,x_flip,y_flip,event_distance,event_angle,event_team,home_team
5259,97.0,-8.0,97.0,-8.0,11.31,45.00,OTT,OTT
51115,58.0,30.0,-58.0,-30.0,43.14,44.06,WSH,WSH
100383,67.0,39.0,67.0,39.0,44.78,60.57,CAR,NYR
366035,-87.0,-7.0,87.0,7.0,7.28,74.05,NYR,OTT
11984,67.0,10.0,-67.0,-10.0,24.17,24.44,PIT,MTL
225579,64.0,39.0,64.0,39.0,46.32,57.34,NSH,PIT
179338,-69.0,-22.0,69.0,22.0,29.73,47.73,WPG,BUF
360347,69.0,22.0,69.0,22.0,29.73,47.73,COL,COL
123160,40.0,-25.0,40.0,-25.0,55.01,27.03,L.A,L.A
260744,71.0,42.0,71.0,42.0,45.69,66.80,WPG,MTL
